In [3]:
import pandas

pandas.set_option("display.max_rows", 20)
df = pandas.read_csv("./1.5.csv")  # Wednesday's changes included
df.sort_values("name", inplace=True)
df.reset_index(inplace=True, drop=True)
df

,name,genres,rating,description,stars,votes,run_time,directors,start_year,end_year
0,13 Reasons Why,"['Drama', 'Mystery', 'Thriller']",6.8,When the dean begins a new investigation and t...,"['Dylan Minnette', 'Grace Saif', 'Christian Na...","1,640",60.0,['Tommy Lohmann'],2017.0,2020.0
1,13 Reasons Why,"['Drama', 'Mystery', 'Thriller']",6.1,The Jensens make the boys take a drug test. Wh...,"['Dylan Minnette', 'Grace Saif', 'Christian Na...","1,507",57.0,['Brenda Strong'],2017.0,2020.0
2,13 Reasons Why,"['Drama', 'Mystery', 'Thriller']",5.7,"After an incident at school, Clay thinks someo...","['Dylan Minnette', 'Grace Saif', 'Christian Na...","1,597",58.0,['Russell Mulcahy'],2017.0,2020.0
3,13 Reasons Why,"['Drama', 'Mystery', 'Thriller']",5.7,"The police question Tyler about the guns, leav...","['Dylan Minnette', 'Grace Saif', 'Christian Na...","1,798",59.0,['Russell Mulcahy'],2017.0,2020.0
4,13 Reasons Why,"['Drama', 'Mystery', 'Thriller']",6.6,"The school goes into lockdown and Clay, Tony a...","['Dylan Minnette', 'Grace Saif', 'Christian Na...","1,694",61.0,['Brenda Strong'],2017.0,2020.0
...,...,...,...,...,...,...,...,...,...,...
9311,Ófærð,"['Crime', 'Drama', 'Mystery']",8.1,"In a remote town in Iceland, Police desperatel...","['Ólafur Darri Ólafsson', 'Ilmur Kristjánsdótt...","17,309",60.0,[],2015.0,NaN
9312,Órbita 9,"['Drama', 'Romance', 'Sci-Fi']",5.9,Helena is a young girl who spent all her life ...,"['Clara Lago', 'Álex González', 'Andrés Parra'...","7,123",95.0,['Hatem Khraiche'],2017.0,2017.0
9313,Ôkami to kôshinryô,"['Animation', 'Adventure', 'Fantasy']",8.0,Kraft Lawrence goes from town to town to make ...,"['Jun Fukuyama', 'Ami Koshimizu', 'Brina Palen...","3,959",24.0,[],2008.0,2009.0
9314,Öldür Beni Sevgilim,"['Comedy', 'Romance']",4.4,Can the excitement of the national lottery sav...,"['Murat Boz', 'Seda Bakan', 'Açelya Topaloglu'...","1,568",104.0,['Senol Sönmez'],2019.0,2019.0


In [4]:
df["name"] = df["name"].str.strip()  # Strip whitespace

In [5]:
df.drop_duplicates(
    inplace=True,
    keep="first",
    subset=[
        "name",
        "genres",
        "rating",
        "stars",
        "votes",
        "run_time",
        "directors",
        "start_year",
        "end_year",
    ],
)
# Check for exact dupes again

In [6]:
groups = df[df.duplicated(["name", "description"], keep=False)].groupby("name")
for key, item in groups:
    print(groups.get_group(key).head(2), "\n\n")
len(groups)

    name                          genres  rating description  \
10  1899  ['Drama', 'History', 'Horror']     NaN         NaN   
11  1899  ['Drama', 'History', 'Horror']     NaN         NaN   

                                                stars votes  run_time  \
10  ['Aneurin Barnard', 'Anton Lesser', 'Mathilde ...     0       NaN   
11                                                 []     0       NaN   

            directors  start_year  end_year  
10  ['Baran bo Odar']      2022.0       NaN  
11  ['Baran bo Odar']      2022.0       NaN   


                         name     genres  rating description  \
77  AlRawabi School for Girls  ['Drama']     NaN         NaN   
78  AlRawabi School for Girls  ['Drama']     NaN         NaN   

                                                stars votes  run_time  \
77  ['Salsabiela A.', 'Joanna Arida', 'Yara Mustaf...     0       NaN   
78  ['Salsabiela A.', 'Joanna Arida', 'Sameera Asi...     0       NaN   

           directors  start_year 

102

In [7]:
df.drop_duplicates(["name", "description"], keep=False, inplace=True)

In [8]:
from ast import literal_eval

from numpy import NaN


def concat(series):
    return (
        series.apply(lambda value: [] if pandas.isna(value) else literal_eval(value))
        .explode()
        .to_list()
    )


def median(series: pandas.Series):
    clean = series.dropna()
    return clean.median() if len(clean) else NaN


for name, group in groups:
    df.append(
        {
            "name": group.name.iloc[0],
            "genres": concat(group.genres),
            "rating": median(group.rating),
            "description": group.description.iloc[0],
            "stars": concat(group.stars),
            "votes": group.votes.sum(),
            "run_time": median(group.run_time),
            "directors": concat(group.directors),
            "start_year": group.start_year.min(),
            "end_year": group.end_year.max(),
        },
        ignore_index=True,
    )  # type: ignore

df.sort_values("name", inplace=True)
df.reset_index(inplace=True, drop=True)

In [9]:
df

,name,genres,rating,description,stars,votes,run_time,directors,start_year,end_year
0,#AnneFrank - Parallel Stories,"['Drama', 'History']",6.5,It is based on five women who did survive the ...,"['Helen Mirren', 'Anne Frank', 'Martina Gatti'...","1,296",92.0,"['Sabina Fedeli', 'Anna Migotto']",2019.0,2019.0
1,#BlackAF,['Comedy'],6.8,A father takes an irreverent and honest approa...,"['Rashida Jones', 'Kenya Barris', 'Iman Benson...","4,200",36.0,[],2020.0,2020.0
2,#HappyBirthdaySense8,['Short'],8.9,NaN,"['Freema Agyeman', 'Jamie Clayton', 'Tina Desa...",828,15.0,['Josh Oreck'],2017.0,2017.0
3,#Realityhigh,"['Comedy', 'Drama', 'Romance']",5.2,High-achieving high-school senior Dani Barnes ...,"['Nesta Cooper', 'Keith Powers', 'Alicia Sanz'...","5,947",99.0,['Fernando Lebrija'],2017.0,2017.0
4,#Rucker50,['Documentary'],5.2,NaN,['Inez Dickens'],105,56.0,['Robert McCullough Jr.'],2016.0,2016.0
...,...,...,...,...,...,...,...,...,...,...
8785,Öldür Beni Sevgilim,"['Comedy', 'Romance']",4.4,Can the excitement of the national lottery sav...,"['Murat Boz', 'Seda Bakan', 'Açelya Topaloglu'...","1,568",104.0,['Senol Sönmez'],2019.0,2019.0
8786,ÜberWeihnachten,"['Comedy', 'Drama', 'Romance']",7.5,Sausages and Potato Salad - Basti hooks up wit...,"['Luke Mockridge', 'Seyneb Saleh', 'Cristina d...",66,50.0,[],2020.0,2020.0
8787,ÜberWeihnachten,"['Comedy', 'Drama', 'Romance']",7.2,The Most Wonderful Christmas of the Year - the...,"['Luke Mockridge', 'Seyneb Saleh', 'Cristina d...",59,45.0,[],2020.0,2020.0
8788,ÜberWeihnachten,"['Comedy', 'Drama', 'Romance']",7.2,"Home Is Where the Tree Is - Bastian, an aspiri...","['Luke Mockridge', 'Seyneb Saleh', 'Cristina d...",65,48.0,[],2020.0,2020.0


In [10]:
df.to_csv("./1.6.csv", index=False)